# Jobs Data Prep

## Libraries

In [1]:
import numpy as np
import pandas as pd
import xlrd
import os
import sys

In [2]:
np.set_printoptions(threshold=np.inf)
pd.options.display.float_format = '{:.4f}'.format

path = 'C:/Users/delightb/Desktop/Final_Project/Senior-Project/Pickle'
####   = pd.read_pickle(os.path.join(path,'abvHouse.pkl'))

# Raw Jobs Data into dataframe


In [ ]:
#xl2 = pd.ExcelFile("/content/drive/My Drive/Colab Notebooks/boise_msa_jobs_2000_to_2018.xlsx")
xl2 = pd.ExcelFile("./Data/boise_msa_jobs_2000_to_2018.xlsx")
rawJobs = xl2.parse("raw")
rawJobs.to_pickle("rawJobs.pkl")

In [ ]:
rawJobs.group.unique()

### From rawJobs.pkl to metro_jobs from pickle

In [ ]:
# open the pickle and turn it into a dataframe
rawJobs = pd.read_pickle("rawJobs.pkl")
metroJobs = rawJobs.copy()

# Splitting Strings
metroJobs['Occ_Cat_Code'], metroJobs['Occ_Sub_Cat_Code'] = metroJobs['occ_code'].str.split('-',1).str


# fixing the ['group'] column
metroJobs['group'] = metroJobs['group'].replace(np.nan,'detailed', regex=True)
metroJobs['occ_titl'] = metroJobs['occ_titl'].replace('Industry Total', 'All Occupations', regex = True)
metroJobs.loc[metroJobs.occ_titl == 'All Occupations', 'group'] = 'total'




# Assigning New Column Names
metroJobs = metroJobs.rename(columns ={'occ_titl': 'Occ_Title', 'group':'Cat_Type',
                                        'tot_emp': 'Emp_Count', 'h_mean':'Hr_Wage_Mean',
                                        'a_mean':'An_Wage_Mean', 'h_median': 'Hr_Median',
                                        'a_median':'An_Median','year':'Year'})



metroJobs['Year'] = metroJobs['Year'].astype(str)
metroJobs['Year'] = metroJobs['Year'].astype('datetime64[Y]')



# Assigning New Data Types
float_list = ['Emp_Count','Hr_Wage_Mean','An_Wage_Mean','Hr_Median','An_Median']

# Cleaning Column Strings for Data Typing
metroJobs[float_list] = metroJobs[float_list].replace('\W','',regex=True)

# Assigning new Data Types
#         need to use floats becuase int cannot handle missing
metroJobs = metroJobs.astype({ 'Cat_Type' : 'category',
                               'Occ_Cat_Code':'category','Occ_Sub_Cat_Code': 'category'})

metroJobs[float_list] = metroJobs[float_list].apply(pd.to_numeric, errors = 'coerce')


# Dropped Columns
metroJobs = metroJobs.drop(['prim_state','area','area_name','occ_code',
                              'emp_prse','mean_prse','h_wpct10','h_wpct25',
                             'h_wpct75','h_wpct90','a_wpct10','a_wpct25',
                             'a_wpct75','a_wpct90'], axis = 1)

# Assigning New Order to the Columns
metroJobs = metroJobs[['Year','Occ_Cat_Code','Occ_Sub_Cat_Code','Occ_Title','Cat_Type',
                        'Emp_Count','Hr_Wage_Mean','An_Wage_Mean','Hr_Median','An_Median']]

metroJobs['Occ_Title'] = metroJobs['Occ_Title'].str.title()
metroJobs['Emp_Count'] = metroJobs['Emp_Count'].fillna(0)
metroJobs['An_Wage_Mean'] = metroJobs['An_Wage_Mean'].fillna(0)



metroJobs.to_pickle('metroJobs.pkl')
metroJobs.head(5)

In [25]:
df = pd.read_pickle(os.path.join(path,'metroJobs.pkl'))

df = df[df['Occ_Title'] == 'Farming, Fishing, And Forestry Occupations']
df

,Year,Occ_Cat_Code,Occ_Sub_Cat_Code,Occ_Title,Cat_Type,Emp_Count,Hr_Wage_Mean,An_Wage_Mean,Hr_Median,An_Median
274,2000-01-01,45,0000,"Farming, Fishing, And Forestry Occupations",major,580.0000,1014.0000,21080.0000,859.0000,17860.0000
689,2001-01-01,45,0000,"Farming, Fishing, And Forestry Occupations",major,710.0000,9.4500,19650.0000,7.1500,14870.0000
1093,2002-01-01,45,0000,"Farming, Fishing, And Forestry Occupations",major,950.0000,9.1900,19110.0000,6.8400,14230.0000
1498,2003-01-01,45,0000,"Farming, Fishing, And Forestry Occupations",major,0.0000,884.0000,18380.0000,694.0000,14440.0000
1925,2004-01-01,45,0000,"Farming, Fishing, And Forestry Occupations",major,830.0000,969.0000,20150.0000,726.0000,15090.0000
2384,2005-01-01,45,0000,"Farming, Fishing, And Forestry Occupations",major,1070.0000,1043.0000,21690.0000,821.0000,17070.0000
2854,2006-01-01,45,0000,"Farming, Fishing, And Forestry Occupations",major,1280.0000,1298.0000,27010.0000,947.0000,19690.0000
3320,2007-01-01,45,0000,"Farming, Fishing, And Forestry Occupations",major,1650.0000,1065.0000,22150.0000,816.0000,16970.0000
3791,2008-01-01,45,0000,"Farming, Fishing, And Forestry Occupations",major,1830.0000,1255.0000,26100.0000,902.0000,18760.0000
4254,2009-01-01,45,0000,"Farming, Fishing, And Forestry Occupations",major,1540.0000,1172.0000,24380.0000,864.0000,17980.0000


# Clean jobs for empolyee count for job type using major job categories

In [26]:
metroJobs = pd.read_pickle(os.path.join(path,'metroJobs.pkl'))

# start filtering to make df with Occ_Title as cols and the value be Emp_Count for each year.

# 0000 is the sub_cat # for the head of Cat
jobsF = metroJobs.copy()

jobsF.at[1498,'Emp_Count'] = ((950+830)/2)

jobsF = jobsF.loc[jobsF['Cat_Type'] != 'detailed']


jobsF['Occ_Title'] = jobsF['Occ_Title'].replace('Services','Service', regex = True)
jobsF['Occ_Title'] = jobsF['Occ_Title'].replace('Practitioners','Practitioner', regex = True)
jobsF['Occ_Title'] = jobsF['Occ_Title'].replace(' Mathematical Science Occupations',' Mathematical Occupations', regex = True)

# value for Employee count for 2003 is 0, which is wrong. Changed for mean between 2002 and 2005


jobsClean = jobsF.copy()
jobsClean.to_pickle(os.path.join(path,'jobsClean.pkl'))
jobsClean.head()
#jobsClean.loc[1498,:]

,Year,Occ_Cat_Code,Occ_Sub_Cat_Code,Occ_Title,Cat_Type,Emp_Count,Hr_Wage_Mean,An_Wage_Mean,Hr_Median,An_Median
0,2000-01-01,11,0000,Management Occupations,major,15790.0000,2950.0000,61360.0000,2668.0000,55490.0000
24,2000-01-01,13,0000,Business And Financial Operations Occupations,major,9560.0000,2150.0000,44710.0000,1942.0000,40390.0000
45,2000-01-01,15,0000,Computer And Mathematical Occupations,major,6020.0000,2577.0000,53600.0000,2377.0000,49440.0000
54,2000-01-01,17,0000,Architecture And Engineering Occupations,major,12860.0000,2178.0000,45300.0000,2062.0000,42900.0000
73,2000-01-01,19,0000,"Life, Physical, And Social Science Occupations",major,2650.0000,1714.0000,35650.0000,1540.0000,32040.0000


## Create employee count % change

In [28]:
df = pd.read_pickle(os.path.join(path,'jobsClean.pkl'))
df = df[['Year','Occ_Title','Emp_Count']]

df['Job_Emp_Change'] = df.groupby('Occ_Title')['Emp_Count'].pct_change()

df = pd.pivot(df, values = 'Job_Emp_Change', index = 'Year', columns = 'Occ_Title')
df = df.reset_index()

df.at[6,'Construction And Extraction Occupations'] = np.nan

df.to_pickle(os.path.join(path,'jobChange.pkl'))
df.head()

Occ_Title,Year,All Occupations,Architecture And Engineering Occupations,"Arts, Design, Entertainment, Sports, And Media Occupations",Building And Grounds Cleaning And Maintenance Occupations,Business And Financial Operations Occupations,Community And Social Service Occupations,Computer And Mathematical Occupations,Construction And Extraction Occupations,"Education, Training, And Library Occupations",...,"Installation, Maintenance, And Repair Occupations",Legal Occupations,"Life, Physical, And Social Science Occupations",Management Occupations,Office And Administrative Support Occupations,Personal Care And Service Occupations,Production Occupations,Protective Service Occupations,Sales And Related Occupations,Transportation And Material Moving Occupations
0,2000-01-01,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,2001-01-01,nan,-0.1135,-0.0916,0.0053,-0.0554,-0.0176,-0.0581,-0.0022,-0.1151,...,-0.0137,-0.0060,-0.2151,0.0165,-0.0204,-0.0424,-0.1350,-0.0052,-0.0250,-0.0455
2,2002-01-01,0.0167,0.0421,0.0444,0.0447,0.0388,0.0717,0.0917,-0.1344,nan,...,0.0224,0.0422,0.0673,-0.0156,0.0474,0.1033,-0.0540,0.0757,0.0420,0.0260
3,2003-01-01,0.0112,-0.4099,-0.0193,-0.0214,-0.0277,0.0936,-0.0905,0.0263,nan,...,0.0836,-0.0231,0.0991,-0.0139,0.0138,0.0769,0.1123,0.0631,0.1000,0.0000
4,2004-01-01,0.0201,0.0428,0.2205,0.0231,0.0559,0.0275,0.1901,-0.0074,0.1806,...,0.0540,0.1006,-0.0615,0.0995,0.0287,-0.0280,-0.1044,0.1553,-0.0134,-0.0431


## Jobs by Industry, employee count

In [32]:
jobsF1 = pd.read_pickle(os.path.join(path,'jobsClean.pkl'))
jobsF1 = jobsF.drop(['Occ_Cat_Code', 'Occ_Sub_Cat_Code', 'Cat_Type'], axis =1)
jobsF1 = jobsF1.iloc[:,0:3]

jobsP1 = pd.pivot(jobsF1, values = 'Emp_Count', index = 'Year', columns = 'Occ_Title')
jobsP1 = jobsP1.reset_index()

# Gives the wrong answer, 2018 should be 706140
jobsP1['Job_Manual_Count'] = jobsP1.iloc[:,2:].sum(axis = 1)


# set manually because the 2000 data did not have a sum of all of the occupations
jobsP1.at[0,'All Occupations'] = 229780

jobsP1.to_pickle(os.path.join(path,'jobTypes.pkl'))
jobsP1.head()

Occ_Title,Year,All Occupations,Architecture And Engineering Occupations,"Arts, Design, Entertainment, Sports, And Media Occupations",Building And Grounds Cleaning And Maintenance Occupations,Business And Financial Operations Occupations,Community And Social Service Occupations,Computer And Mathematical Occupations,Construction And Extraction Occupations,"Education, Training, And Library Occupations",...,Legal Occupations,"Life, Physical, And Social Science Occupations",Management Occupations,Office And Administrative Support Occupations,Personal Care And Service Occupations,Production Occupations,Protective Service Occupations,Sales And Related Occupations,Transportation And Material Moving Occupations,Job_Manual_Count
0,2000-01-01,229780.0000,12860.0000,2730.0000,7570.0000,9560.0000,2840.0000,6020.0000,13650.0000,10950.0000,...,1670.0000,2650.0000,15790.0000,38160.0000,2830.0000,19040.0000,3850.0000,21960.0000,14930.0000,229780.0000
1,2001-01-01,222100.0000,11400.0000,2480.0000,7610.0000,9030.0000,2790.0000,5670.0000,13620.0000,9690.0000,...,1660.0000,2080.0000,16050.0000,37380.0000,2710.0000,16470.0000,3830.0000,21410.0000,14250.0000,222100.0000
2,2002-01-01,225820.0000,11880.0000,2590.0000,7950.0000,9380.0000,2990.0000,6190.0000,11790.0000,nan,...,1730.0000,2220.0000,15800.0000,39150.0000,2990.0000,15580.0000,4120.0000,22310.0000,14620.0000,216720.0000
3,2003-01-01,228350.0000,7010.0000,2540.0000,7780.0000,9120.0000,3270.0000,5630.0000,12100.0000,nan,...,1690.0000,2440.0000,15580.0000,39690.0000,3220.0000,17330.0000,4380.0000,24540.0000,14620.0000,216630.0000
4,2004-01-01,232950.0000,7310.0000,3100.0000,7960.0000,9630.0000,3360.0000,6700.0000,12010.0000,11440.0000,...,1860.0000,2290.0000,17130.0000,40830.0000,3130.0000,15520.0000,5060.0000,24210.0000,13990.0000,232950.0000


## Clean jobs for empolyee count for Salery Range not using major job categories

## Jobs by Salery Range, empolyee count

In [34]:
df = pd.read_pickle(os.path.join(path,'metroJobs.pkl'))

df = df.loc[df['Cat_Type'] == 'detailed']
df = df[['Year', 'Emp_Count' ,'An_Wage_Mean']]

jobsSalery = df.groupby([df.Year,pd.cut(df.An_Wage_Mean,[10000,40000,70000,100000,150000,200000,300000]).astype(str)])[['Emp_Count']].sum().reset_index()

jobsSalery = pd.pivot(jobsSalery, values = 'Emp_Count', index = 'Year', columns = 'An_Wage_Mean')
jobsSalery = jobsSalery.reset_index()
jobsSalery = jobsSalery.replace(np.nan,0)

# fix the order of the columns
jobsSalery = jobsSalery[['Year','(10000.0, 40000.0]','(40000.0, 70000.0]','(70000.0, 100000.0]',
          '(100000.0, 150000.0]', '(150000.0, 200000.0]', '(200000.0, 300000.0]' ]]

jobsSalery.to_pickle(os.path.join(path,'jobSalery.pkl'))
jobsSalery

An_Wage_Mean,Year,"(10000.0, 40000.0]","(40000.0, 70000.0]","(70000.0, 100000.0]","(100000.0, 150000.0]","(150000.0, 200000.0]","(200000.0, 300000.0]"
0,2000-01-01,150080.0000,34980.0000,3710.0000,0.0000,0.0000,0.0000
1,2001-01-01,135860.0000,45570.0000,4520.0000,0.0000,0.0000,0.0000
2,2002-01-01,129450.0000,40330.0000,5850.0000,1100.0000,0.0000,0.0000
3,2003-01-01,132290.0000,40650.0000,4960.0000,1160.0000,0.0000,0.0000
4,2004-01-01,143120.0000,44460.0000,5710.0000,2030.0000,0.0000,0.0000
5,2005-01-01,147750.0000,53520.0000,4010.0000,3070.0000,220.0000,0.0000
6,2006-01-01,159950.0000,50600.0000,9800.0000,3970.0000,200.0000,0.0000
7,2007-01-01,167110.0000,60680.0000,20230.0000,3950.0000,240.0000,0.0000
8,2008-01-01,162520.0000,64660.0000,22890.0000,4680.0000,250.0000,0.0000
9,2009-01-01,146070.0000,65420.0000,23110.0000,1280.0000,230.0000,590.0000


## Percent change in # of emp for wage range

In [35]:
df = pd.read_pickle(os.path.join(path,'metroJobs.pkl'))

df = df.loc[df['Cat_Type'] == 'detailed']
df = df[['Year', 'Emp_Count' ,'An_Wage_Mean']]

df = df.groupby([df.Year,pd.cut(df.An_Wage_Mean,[10000,40000,70000,100000,150000,200000,
                                                 300000]).astype(str)])[['Emp_Count']].sum().reset_index()

df['Salery_Count_Change'] = df.groupby('An_Wage_Mean')['Emp_Count'].pct_change()

df = pd.pivot(df, values = 'Salery_Count_Change', index = 'Year', columns = 'An_Wage_Mean')
df = df.reset_index()

# These are the first values for that range, nothing to compare against so NaN
df.at[5, '(150000.0, 200000.0]'] = np.nan
df.at[9, '(200000.0, 300000.0]'] = np.nan

# Not sure what to do when values go from 220 - 0 and 0 - 220, setting to NaN for now.
#df.at[16, '(150000.0, 200000.0]'] = np.nan
df.at[18, '(150000.0, 200000.0]'] = np.nan

df = df[['Year','(10000.0, 40000.0]','(40000.0, 70000.0]','(70000.0, 100000.0]',
          '(100000.0, 150000.0]', '(150000.0, 200000.0]', '(200000.0, 300000.0]' ]]

df.to_pickle(os.path.join(path,'salChange.pkl'))
df

An_Wage_Mean,Year,"(10000.0, 40000.0]","(40000.0, 70000.0]","(70000.0, 100000.0]","(100000.0, 150000.0]","(150000.0, 200000.0]","(200000.0, 300000.0]"
0,2000-01-01,nan,nan,nan,nan,nan,nan
1,2001-01-01,-0.0947,0.3027,0.2183,nan,nan,nan
2,2002-01-01,-0.0472,-0.1150,0.2942,nan,nan,nan
3,2003-01-01,0.0219,0.0079,-0.1521,0.0545,nan,nan
4,2004-01-01,0.0819,0.0937,0.1512,0.7500,nan,nan
5,2005-01-01,0.0324,0.2038,-0.2977,0.5123,nan,nan
6,2006-01-01,0.0826,-0.0546,1.4439,0.2932,-0.0909,nan
7,2007-01-01,0.0448,0.1992,1.0643,-0.0050,0.2000,nan
8,2008-01-01,-0.0275,0.0656,0.1315,0.1848,0.0417,nan
9,2009-01-01,-0.1012,0.0118,0.0096,-0.7265,-0.0800,nan


In [36]:
job1 = pd.read_pickle(os.path.join(path,'jobTypes.pkl'))
job2 = pd.read_pickle(os.path.join(path,'jobSalery.pkl'))

jobFeatures = pd.merge(job1, job2, on = 'Year', how = 'left')
jobFeatures = jobFeatures.drop(['Job_Manual_Count'], axis = 1)

jobFeatures.to_pickle(os.path.join(path,'jobFeatures.pkl'))
jobFeatures.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19 entries, 0 to 18
Data columns (total 30 columns):
Year                                                          19 non-null datetime64[ns]
All Occupations                                               19 non-null float64
Architecture And Engineering Occupations                      19 non-null float64
Arts, Design, Entertainment, Sports, And Media Occupations    19 non-null float64
Building And Grounds Cleaning And Maintenance Occupations     19 non-null float64
Business And Financial Operations Occupations                 19 non-null float64
Community And Social Service Occupations                      19 non-null float64
Computer And Mathematical Occupations                         19 non-null float64
Construction And Extraction Occupations                       19 non-null float64
Education, Training, And Library Occupations                  17 non-null float64
Farming, Fishing, And Forestry Occupations                    19 non-n

In [37]:
df1 = pd.read_pickle(os.path.join(path,'jobChange.pkl'))
df2 = pd.read_pickle(os.path.join(path,'salChange.pkl'))

df = pd.merge(df1, df2, on = 'Year', how = 'left')

# replace nan values in salery columns with 0
df['(150000.0, 200000.0]'].fillna(0, inplace = True)
df['(200000.0, 300000.0]'].fillna(0, inplace = True)
df['(100000.0, 150000.0]'].fillna(0, inplace = True)

df.fillna(df.mean(), inplace=True)

df.to_pickle(os.path.join(path,'jobShift.pkl'))
df


,Year,All Occupations,Architecture And Engineering Occupations,"Arts, Design, Entertainment, Sports, And Media Occupations",Building And Grounds Cleaning And Maintenance Occupations,Business And Financial Operations Occupations,Community And Social Service Occupations,Computer And Mathematical Occupations,Construction And Extraction Occupations,"Education, Training, And Library Occupations",...,Production Occupations,Protective Service Occupations,Sales And Related Occupations,Transportation And Material Moving Occupations,"(10000.0, 40000.0]","(40000.0, 70000.0]","(70000.0, 100000.0]","(100000.0, 150000.0]","(150000.0, 200000.0]","(200000.0, 300000.0]"
0,2000-01-01,0.0227,-0.0188,0.0346,0.0187,0.0251,0.0317,0.0243,-0.0616,0.0283,...,0.0073,0.0272,0.0234,0.0177,0.0107,0.0577,0.1714,0.0000,0.0000,0.0000
1,2001-01-01,0.0227,-0.1135,-0.0916,0.0053,-0.0554,-0.0176,-0.0581,-0.0022,-0.1151,...,-0.1350,-0.0052,-0.0250,-0.0455,-0.0947,0.3027,0.2183,0.0000,0.0000,0.0000
2,2002-01-01,0.0167,0.0421,0.0444,0.0447,0.0388,0.0717,0.0917,-0.1344,0.0283,...,-0.0540,0.0757,0.0420,0.0260,-0.0472,-0.1150,0.2942,0.0000,0.0000,0.0000
3,2003-01-01,0.0112,-0.4099,-0.0193,-0.0214,-0.0277,0.0936,-0.0905,0.0263,0.0283,...,0.1123,0.0631,0.1000,0.0000,0.0219,0.0079,-0.1521,0.0545,0.0000,0.0000
4,2004-01-01,0.0201,0.0428,0.2205,0.0231,0.0559,0.0275,0.1901,-0.0074,0.1806,...,-0.1044,0.1553,-0.0134,-0.0431,0.0819,0.0937,0.1512,0.7500,0.0000,0.0000
5,2005-01-01,0.0918,0.0561,-0.0774,-0.0427,0.0706,-0.0625,0.0582,-1.0000,0.0411,...,0.1604,-0.0534,0.0302,0.0636,0.0324,0.2038,-0.2977,0.5123,0.0000,0.0000
6,2006-01-01,0.0561,0.1593,0.1573,0.1142,0.1290,0.1111,-0.0169,-0.0616,-0.0042,...,0.0050,-0.0960,0.0505,0.1055,0.0826,-0.0546,1.4439,0.2932,-0.0909,0.0000
7,2007-01-01,0.0579,-0.0525,0.3384,0.0718,-0.0129,0.1886,0.0516,0.0209,0.0152,...,0.1105,0.1155,0.1462,0.1812,0.0448,0.1992,1.0643,-0.0050,0.2000,0.0000
8,2008-01-01,0.0068,0.0212,0.0451,-0.0396,-0.0044,0.3173,-0.0559,-0.1956,-0.0008,...,-0.0294,0.1863,0.0460,-0.0427,-0.0275,0.0656,0.1315,0.1848,0.0417,0.0000
9,2009-01-01,-0.0771,-0.0820,0.0713,-0.0458,-0.0664,0.0000,0.0954,-0.1917,0.0466,...,-0.2573,-0.0558,-0.0818,-0.0527,-0.1012,0.0118,0.0096,-0.7265,-0.0800,0.0000


In [ ]:
df = pd.read_pickle(os.path.join(path,'jobShift.pkl'))
df.head()
#print(df.isnull().sum())